In [2]:
# STEP 1: Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
import tensorflow as tf



In [3]:
# STEP 2: Load and Prepare Dataset
def load_dataset(data_dir):
    image_paths = []
    labels = []
    for label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, label)
        if os.path.isdir(class_dir):
            for img_file in os.listdir(class_dir):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_paths.append(os.path.join(class_dir, img_file))
                    labels.append(label)
    return pd.DataFrame({'image_path': image_paths, 'label': labels})

train_dir = '/kaggle/input/train-dataset'
test_dir = '/kaggle/input/test-dataset'

train_df = load_dataset(train_dir)
test_df = load_dataset(test_dir)

print("Train size:", len(train_df), "| Test size:", len(test_df))
print(train_df['label'].value_counts())


Train size: 528 | Test size: 222
label
Eczemaa                 88
Acne                    88
Rosacea                 88
normal                  88
Actinic Keratosis       88
Basal Cell Carcinoma    88
Name: count, dtype: int64


In [4]:
# STEP 3: Data Generator Setup (Stronger Augmentation)
IMG_SIZE = 224
BATCH_SIZE = 32

datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=(0.8, 1.2)
)

train_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_dataframe(
    train_df,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset='validation',
    shuffle=True
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = test_datagen.flow_from_dataframe(
    test_df,
    x_col='image_path',
    y_col='label',
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    batch_size=1,
    shuffle=False
)

Found 476 validated image filenames belonging to 6 classes.
Found 52 validated image filenames belonging to 6 classes.
Found 222 validated image filenames belonging to 6 classes.


In [6]:
# STEP 4: Build the Model with Regularization
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.4)(x)
num_classes = len(train_gen.class_indices)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False  # Freeze initial layers

model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_4          │ (None, 230, 230, 3)    │              0 │ input_layer_2[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ zero_padding2d_4[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_5          │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 56, 56, 64)     │              0 │ zero_padding2d_5[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 56, 56, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 56, 56, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├──────────────────────

 Total params: 7,173,574 (27.37 MB)

 Trainable params: 134,022 (523.52 KB)

 Non-trainable params: 7,039,552 (26.85 MB)

In [8]:
# STEP 5: Train the Model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint("best_skin_model.h5", monitor='val_accuracy', save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, verbose=1)
]

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.2875 - loss: 2.2449 - val_accuracy: 0.0000e+00 - val_loss: 2.9963 - learning_rate: 5.0000e-04
Epoch 2/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.5043 - loss: 1.6530 - val_accuracy: 0.0000e+00 - val_loss: 3.2825 - learning_rate: 5.0000e-04
Epoch 3/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5695 - loss: 1.4302
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
15/15 ━━━━━━━━━━━━━━━━━━━━ 61s 4s/step - accuracy: 0.5705 - loss: 1.4254 - val_accuracy: 0.0577 - val_loss: 3.3136 - learning_rate: 5.0000e-04
Epoch 4/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.6265 - loss: 1.2569 - val_accuracy: 0.0769 - val_loss: 3.1090 - learning_rate: 1.0000e-04
Epoch 5/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.6633 - loss: 1.0666 - val_accuracy: 0.1154 - val_loss: 2.8118 - learning_rate: 1.0000e-04
Epoch 6/20
15/15 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.6479 - lo

In [9]:
# STEP 6: Fine-Tune Top Layers (Optional)
# Unfreeze last 30 layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 94s 4s/step - accuracy: 0.6705 - loss: 1.1102 - val_accuracy: 0.4615 - val_loss: 1.9944 - learning_rate: 1.0000e-05
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.6672 - loss: 1.0642 - val_accuracy: 0.3654 - val_loss: 1.9503 - learning_rate: 1.0000e-05
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.7062 - loss: 1.0416 - val_accuracy: 0.3654 - val_loss: 1.9585 - learning_rate: 1.0000e-05
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.6677 - loss: 1.0417 - val_accuracy: 0.3269 - val_loss: 1.8740 - learning_rate: 1.0000e-05
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 57s 4s/step - accuracy: 0.7181 - loss: 1.0438 - val_accuracy: 0.4423 - val_loss: 1.7839 - learning_rate: 1.0000e-05
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 56s 4s/step - accuracy: 0.6979 - loss: 0.9993 - val_accuracy: 0.4423 - val_loss: 1.9236 - learning_rate: 1.0000e-05
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6826 - loss: 

In [10]:
# STEP 7: Evaluate the Model
preds = model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)
y_true = test_gen.classes

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys())))


222/222 ━━━━━━━━━━━━━━━━━━━━ 31s 114ms/step
Classification Report:
                      precision    recall  f1-score   support

                Acne       0.61      0.89      0.73        37
   Actinic Keratosis       0.83      0.51      0.63        37
Basal Cell Carcinoma       0.61      0.73      0.67        37
             Eczemaa       0.79      0.51      0.62        37
             Rosacea       0.61      0.62      0.61        37
              normal       0.95      1.00      0.97        37

            accuracy                           0.71       222
           macro avg       0.73      0.71      0.71       222
        weighted avg       0.73      0.71      0.71       222



In [11]:
# STEP 8: Predict on Single Image
def predict_image(path):
    img = Image.open(path).resize((IMG_SIZE, IMG_SIZE))
    img_array = preprocess_input(np.expand_dims(np.array(img), axis=0))
    prediction = model.predict(img_array)
    class_label = list(train_gen.class_indices.keys())[np.argmax(prediction)]
    confidence = np.max(prediction)
    print(f"Predicted class: {class_label} ({confidence:.2f})")


In [12]:
predict_image('/kaggle/input/test-dataset/Eczemaa/10_eczema-lids-9.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
Predicted class: Basal Cell Carcinoma (0.32)


In [13]:
predict_image('/kaggle/input/test-dataset/Acne/40_07AcnePittedScars1.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
Predicted class: Acne (0.49)


In [14]:
predict_image('/kaggle/input/test-dataset/normal/188_Selfie_12.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
Predicted class: normal (0.99)


In [15]:
predict_image('/kaggle/input/test-dataset/Basal Cell Carcinoma/155_basal-cell-carcinoma-lid-22.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
Predicted class: Basal Cell Carcinoma (0.62)


In [16]:

# STEP 9: Save the Final Trained Model
model.save("final_skin_disease_model.h5")
print("✅ Final model saved as 'final_skin_disease_model.h5'")    


✅ Final model saved as 'final_skin_disease_model.h5'
